In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dynamic_routing_analysis import decoding_utils, plot_utils
import dynamic_routing_analysis as dra

import matplotlib
import matplotlib.font_manager as fm

matplotlib.rcParams['font.size'] = 8
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
fm.FontProperties().set_family('arial')

%load_ext autoreload
%autoreload 2
%matplotlib inline
# %matplotlib widget

In [2]:
#set path to decoding results
# loadpath=dra.DECODING_ROOT_PATH / "new_annotations" / "linear_shift_20_units_re_run"
# loadpath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_20_units_plus_all"
loadpath=r"\\allen\programs\mindscope\workgroups\dynamicrouting\shailaja\population_decoding"
files=glob.glob(os.path.join(loadpath,'*_decoding_results*.pkl'))

## plot single session linear shift decoder results

In [ ]:
files[90]

In [ ]:
#plot decoding results from one session
sel_session=90

plot_utils.plot_single_session_decoding_results(path=files[sel_session])

## load or concat decoder results across sessions

In [3]:
# load all decoding results, option to concatenate all sessions
concat_session_results=False
save_table=False
return_table=False

# all_decoder_results_path=os.path.join(loadpath,'combined','all_unit_linear_shift_use_more_trials.csv')
# all_decoder_results_path=loadpath / 'combined' / 'all_unit_linear_shift_use_more_trials.csv'

all_decoder_results_path=r"\\allen\programs\mindscope\workgroups\dynamicrouting\shailaja\population_decoding\all_unit_linear_shift_use_more_trials.csv"

if save_table:
    # savepath = (
    #     dra.DECODING_ROOT_PATH
    #     / "new_annotations"
    #     / "linear_shift_20_units_re_run"
    #     / "combined"
    # )
    savepath=all_decoder_results_path=os.path.join(loadpath,'combined')
else:
    savepath=None

if concat_session_results:
    all_decoder_results=decoding_utils.concat_decoder_results(files,savepath=savepath,return_table=return_table)
else:
    all_decoder_results=pd.read_csv(all_decoder_results_path)

In [ ]:
all_decoder_results

In [4]:
#compute fraction significant, difference from null for DynamicRouting and Templeton projects
all_frac_sig_df,all_diff_from_null_df=decoding_utils.compute_significant_decoding_by_area(all_decoder_results)

## plot decoding accuracy across areas for DR sessions

In [ ]:
#barplot sorted by fraction significant
min_n_expts=3

n_units=[]
for col in all_frac_sig_df.filter(like='frac_sig_DR').columns.values:
    if len(col.split('_'))==4:
        n_units.append('_'+col.split('_')[3])
    else:
        n_units.append('')

for nu in n_units:
    plot_DR_frac_sig_df=all_frac_sig_df.query('n_expts_DR>=@min_n_expts').sort_values('frac_sig_DR'+nu,ascending=False)
    fig,ax=plt.subplots(1,1,figsize=(10,4))
    plot_DR_frac_sig_df.plot.bar(x='area',y=['frac_sig_DR'+nu],ax=ax)
    ax.set_ylabel('fraction significant decoding')

    area_labels=[]
    for i in range(plot_DR_frac_sig_df.shape[0]):
        area_labels.append(plot_DR_frac_sig_df['area'].iloc[i]+' ('+str(int(plot_DR_frac_sig_df['n_expts_DR'].iloc[i]))+')')
    ax.set_xticklabels(area_labels,rotation=90,ha='center')
    ax.set_ylim([0,1])
    fig.tight_layout()

# plot_DR_frac_sig_df=all_frac_sig_df.query('n_expts_DR>=@min_n_expts').sort_values('frac_sig_DR',ascending=False)
# fig,ax=plt.subplots(1,1,figsize=(8,4))
# plot_DR_frac_sig_df.plot.bar(x='area',y=['frac_sig_DR'],ax=ax)
# ax.set_ylabel('fraction significant decoding')

# area_labels=[]
# for i in range(plot_DR_frac_sig_df.shape[0]):
#     area_labels.append(plot_DR_frac_sig_df['area'].iloc[i]+' ('+str(int(plot_DR_frac_sig_df['n_expts_DR'].iloc[i]))+')')
# ax.set_xticklabels(area_labels,rotation=90,ha='center')
# ax.set_ylim([0,1])
# fig.tight_layout()

In [ ]:
#barplot sorted by DR diff from median
min_n_expts=3

n_units=[]
for col in all_frac_sig_df.filter(like='frac_sig_DR').columns.values:
    if len(col.split('_'))==4:
        n_units.append('_'+col.split('_')[3])
    else:
        n_units.append('')

for nu in n_units[0:1]:
    plot_DR_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR'+nu,ascending=False)
    fig,ax=plt.subplots(1,1,figsize=(10,4))
    plot_DR_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR'+nu],
                                       yerr='diff_from_null_sem_DR'+nu,ax=ax)

    ax.set_ylabel('median difference from null')
    area_labels=[]
    for i in range(plot_DR_diff_from_null_df.shape[0]):
        area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(int(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i]))+')')
    ax.set_xticklabels(area_labels,rotation=90,ha='center')

    ax.set_ylim([0,0.17])
    fig.tight_layout()

# plot_DR_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
# fig,ax=plt.subplots(1,1,figsize=(8,4))
# plot_DR_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR'],ax=ax)

# ax.set_ylabel('median difference from null')
# area_labels=[]
# for i in range(plot_DR_diff_from_null_df.shape[0]):
#     area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(int(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i]))+')')
# ax.set_xticklabels(area_labels,rotation=90,ha='center')

# ax.set_ylim([0,0.17])
# fig.tight_layout()



In [ ]:
#plot only selected areas

#lump SC motor areas together?
min_n_expts=3

selected_areas=['SCiw','MRN','FRP','AId','CP','ACAd','PL','MOs','ORBl','VISal','MOp','MD','AUDp','MGd','VISp','LGd','SCsg']

for nu in n_units[0:1]:
    plot_DR_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR'+nu,ascending=False)
    plot_DR_diff_from_null_df=plot_DR_diff_from_null_df[plot_DR_diff_from_null_df['area'].isin(selected_areas)]
    fig,ax=plt.subplots(1,1,figsize=(4,2.3))
    plot_DR_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR'+nu],
                                       yerr='diff_from_null_sem_DR'+nu,ax=ax,legend=False,color='k')

    ax.set_ylabel('decoding accuracy \n above null')
    area_labels=[]
    for i in range(plot_DR_diff_from_null_df.shape[0]):
        area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(int(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i]))+')')
    ax.set_xticklabels(area_labels,rotation=90,ha='center')

    ax.set_ylim([0,0.17])
    ax.set_xlabel('')
    #remove right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    fig.tight_layout()


In [10]:
# #save figure
# temp_loadpath=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\quick figures\2024-10-04-plots_for_paper"
# savepath=os.path.join(temp_loadpath,'selected_areas_diff_from_null_.pdf')
# # fig.savefig(savepath,format='pdf',bbox_inches='tight',dpi=600)
# fig.savefig(savepath,format='pdf',transparent=True, bbox_inches='tight', pad_inches=0,
#             metadata=None, dpi=600)

In [ ]:
plot_DR_diff_from_null_df.query('area=="MRN"')

In [ ]:
#plot true accuracy and null median on same barplot

min_n_expts=3

plot_DR_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(10,4))
plot_DR_diff_from_null_df.plot.bar(x='area',y=['true_accuracy_DR','null_median_DR'],ax=ax)

ax.set_ylabel('accuracy')
area_labels=[]
for i in range(plot_DR_diff_from_null_df.shape[0]):
    area_labels.append(plot_DR_diff_from_null_df['area'].iloc[i]+' ('+str(int(plot_DR_diff_from_null_df['n_expts_DR'].iloc[i]))+')')

ax.set_xticklabels(area_labels,rotation=90,ha='center')

ax.set_ylim([0.5,1.0])
fig.tight_layout()

## compare DR and Templeton

In [ ]:
#barplot sorted by fraction significant
min_n_expts=2

plot_all_frac_sig_df=all_frac_sig_df.query('n_expts_DR>=@min_n_expts and n_expts_Templ>=@min_n_expts').sort_values('frac_sig_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_all_frac_sig_df.plot.bar(x='area',y=['frac_sig_DR','frac_sig_Templ'],ax=ax)
ax.set_ylabel('fraction significant decoding')
# ax.set_xlabel('area')
ax.legend(['DR','Templ'])
ax.set_ylim([0,1])
fig.tight_layout()

In [ ]:
#barplot sorted by DR diff from median
min_n_expts=2

plot_all_diff_from_null_df=all_diff_from_null_df.query('n_expts_DR>=@min_n_expts and n_expts_Templ>=@min_n_expts').sort_values('diff_from_null_median_DR',ascending=False)
fig,ax=plt.subplots(1,1,figsize=(8,4))
plot_all_diff_from_null_df.plot.bar(x='area',y=['diff_from_null_median_DR','diff_from_null_median_Templ'],ax=ax)

ax.set_ylabel('median difference from null')
# ax.set_xlabel('area')
ax.legend(['DR','Templ'])

ax.set_ylim([-0.03,0.18])
fig.tight_layout()


### compare 3 areas - DR

In [ ]:
area1='MRN'
area2='MOs'
area3='VISp'

x_spacing=1

for nu in n_units:

    fig,ax=plt.subplots(1,1,figsize=(2.5,2.8))

    area1_df=all_decoder_results.query('project=="DynamicRouting" and area==@area1 and n_good_blocks>=4')[['true_accuracy'+nu,'null_accuracy_median'+nu]]
    area1_values=np.vstack(area1_df.values)
    area1_mean=area1_values.mean(axis=0)

    area2_df=all_decoder_results.query('project=="DynamicRouting" and area==@area2 and n_good_blocks>=4')[['true_accuracy'+nu,'null_accuracy_median'+nu]]
    area2_values=np.vstack(area2_df.values)
    area2_mean=area2_values.mean(axis=0)

    area3_df=all_decoder_results.query('project=="DynamicRouting" and area==@area3 and n_good_blocks>=4')[['true_accuracy'+nu,'null_accuracy_median'+nu]]
    area3_values=np.vstack(area3_df.values)
    area3_mean=area3_values.mean(axis=0)


    ax.plot([0,1],area1_values.T,'-',linewidth=0.5,color='gray',alpha=0.5)
    ax.plot([0,1],area1_mean,'ko-')
    ax.plot([1.75,2.75],area2_values.T,'-',linewidth=0.5,color='gray',alpha=0.5)
    ax.plot([1.75,2.75],area2_mean,'ko-')
    ax.plot([3.5,4.5],area3_values.T,'-',linewidth=0.5,color='gray',alpha=0.5)
    ax.plot([3.5,4.5],area3_mean,'ko-')

    ax.set_xticks([0.1,0.9,1.75+0.1,2.75-0.1,3.5+0.1,4.5-0.1])
    ax.set_xticklabels(['obs.','null','obs.','null','obs.','null'])

    #label axis with areas
    ax.set_ylabel('decoder accuracy')

    # ax.set_title(area1+' vs. '+area2+' ('+nu+' units)')

    #remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    fig.tight_layout()

In [12]:
# #save figure
# savepath=r"C:\Users\ethan.mcbride\OneDrive - Allen Institute\quick figures\2024-10-04-plots_for_paper"
# fig.savefig(os.path.join(savepath,area1+'_'+area2+'_accuracy_comparison_.pdf'),format='pdf',bbox_inches='tight',dpi=600)